In [ ]:
## This script uses the invoice_detail function to load a Customer Invoice that comes from accounting
## Running this in a loop makes it easy to load a year's worth of invoices and show trends

In [1]:
import glob, numpy as np, pandas as pd

In [2]:
def invoice_detail(path, invoice_sheet='Invoice', detail_sheet='Detail', columns=[1, 2, 3, 5, 6],
                   column_names=['detail1', 'detail2', 'detail3', 'general_ledger', 'invoice']):
    """Open invoice file and return pandas DataFrame with added date and dept columns."""
    
    # Creating an array from the first sheet to search for the invoice date
    arr = (np.array(pd.read_excel(path, sheet_name=invoice_sheet, usecols=list(range(4)))
     [lambda x: x.apply(lambda r: r.str.contains('invoice date', case=False).any(), axis=1)])).flatten()

    # Finding invoice date
    for i in arr:
        if isinstance(i, pd.datetime) == False:
            pass
        else:
            date = i
            break
    
    # Loading file from defined path name
    df = (pd.read_excel(path,
                    sheet_name=detail_sheet,
                    header=None,
                    names=column_names,
                    usecols=columns)
      .dropna(thresh=1))
    
    detail_col = df.columns[0]
    invoice_depts = list(df[detail_col].loc[df[df[detail_col] == 'Invoice'].index - 1]) + ['Contracted Charges']
    
    filter_idx = []
    for i in list(range(len(df[detail_col]))):
        if df[detail_col].iloc[i] == 'Invoice':
            j = i + 1
            while df[detail_col].iloc[j] != 'Total':
                filter_idx.append(j)
                j += 1
        elif df[detail_col].iloc[i] == 'Contracted Charges':
            j = i + 1
            while df[detail_col].iloc[j] != 'Total Operating Expense':
                filter_idx.append(j)
                j += 1
            break
    
    df['dept'] = list(pd.Series(np.where(df[detail_col].isin(invoice_depts), df[detail_col], np.nan)).fillna(method='ffill'))

    df['date'] = pd.Period(date, 'M') - 1

    return df.iloc[filter_idx]

In [3]:
def invoice_cost_center(path, invoice_sheet='Invoice', detail_sheet='Detail', columns=[1, 3, 6],
                   column_names=['description', 'cost_center_code', 'invoice']):
    """Finding cost center code dollar values from invoice."""
    
    # Creating an array from the first sheet to search for the invoice date
    arr = (np.array(pd.read_excel(path, sheet_name=invoice_sheet, usecols=list(range(4)))
     [lambda x: x.apply(lambda r: r.str.contains('invoice date', case=False).any(), axis=1)])).flatten()

    # Finding invoice date
    for i in arr:
        if isinstance(i, pd.datetime) == False:
            pass
        else:
            date = i
            break
    
    # Loading file from defined path name
    df = (pd.read_excel(path,
                    sheet_name=detail_sheet,
                    header=None,
                    names=column_names,
                    usecols=columns)
      .dropna(thresh=1))
    
    detail_col = df.columns[0]
    cost_center_code_col = df.columns[1]
    invoice_depts = df[detail_col].loc[df[df[detail_col] == 'Invoice'].index - 1]
    
    filter_idx = []
    for i in list(range(len(df.iloc[:, 2]))):
        if df.iloc[i, 2] == 'Invoice':
            filter_idx.append(i + 1)

    
    df[cost_center_code_col].fillna(method='ffill', inplace=True)
    df['dept'] = list(pd.Series(np.where(df[detail_col].isin(invoice_depts), df[detail_col], np.nan)).fillna(method='ffill'))
    df['date'] = pd.Period(date, 'M') - 1

    return df.iloc[filter_idx, [4, 3, 1, 2]]

In [4]:
# Invoice Groupings of Departments

total_fixed_personnel_depts = ['Supervision', 'Supervision Incentive',
       'Supervision Fringes @ 34.90%', 'Clerical', 'Clerical OT',
       'Clerical Fringes @ 32.56%', 'Indirect Labor',
       'Indirect Labor O.T.', 'Indirect Vacation/Holiday ',
       'Indirect Fringes @ 44.61% ST, 18.77% OT']

total_variable_personnel_depts = ['Warehouse Labor',
       'Warehouse O.T.', 'Warehouse Vacation/Holiday',
       'Warehouse Fringes @ 47.36% ST, 18.77% OT', 'Temporary Labor']

total_operating_depts = ['Office Supplies ', 'Postage', 'Warehouse Supplies',
       'Maintenance - Warehouse + WH Equipment',
       'Computer Hardware/Software Maintenance', 'Misc Expense ',
       'Communications', 'Employment Related Professional Fees',
       'Insurance', 'Travel Expense', 'Amortization Expense',
       'Equipment Rental', 'Janitorial Expense']

markup_dictionary = {**{f:0.08 for f in total_fixed_personnel_depts},
 **{f:0.08 for f in total_variable_personnel_depts},
 **{f:0.04 for f in total_operating_depts}}

dept_category_order = (total_fixed_personnel_depts + total_variable_personnel_depts +
                       total_operating_depts +
              ['Contracted Charges', 'personnel_expense_markup', 'operating_expense_markup'])

In [5]:
file_path = r'my_user_name_path\invoices\customer_invoice\fender_invoice_'
file_list = glob.glob(file_path + '*.xlsm')

In [6]:
combined_invoice_detail = pd.concat([invoice_detail(f) for f in file_list], ignore_index=True)

In [7]:
combined_cost_center = pd.concat([invoice_cost_center(f) for f in file_list], ignore_index=True)
combined_cost_center['split_cost_center_codes'] = combined_cost_center.cost_center_code.apply(lambda x: tuple(str(x).split(',')))

In [8]:
combined_cost_center.head()

,date,dept,cost_center_code,invoice,split_cost_center_codes
0,2018-01,Supervision,540100,21518.3,"(540100,)"
1,2018-01,Supervision Incentive,548050,2689.82,"(548050,)"
2,2018-01,Supervision Fringes @ 34.90%,Calc,8448.62,"(Calc,)"
3,2018-01,Clerical,"520242, 581203",14030.7,"(520242, 581203)"
4,2018-01,Clerical OT,554173,1272.43,"(554173,)"


In [ ]:
## Some testing of loops and data exploration

In [9]:
# I want a function or bit of code that will split dollar values in the invoice
# and assign them to each individual cost center code.
# This will allow me to match these cost center codes and dollar values to the Consumer FY20 Budget Plan file.

In [10]:
# Divide total department cost from invoice files to individual cost centers.

ls = []

for idx, val in enumerate(combined_cost_center.split_cost_center_codes):
    #print(idx, combined_cost_center.invoice.iloc[idx], val)
    for i in val:
        #print(idx, combined_cost_center.date.iloc[idx], combined_cost_center.invoice.iloc[idx], len(val), combined_cost_center.invoice.iloc[idx]/len(val), i)
        ls.append([idx, combined_cost_center.date.iloc[idx], combined_cost_center.invoice.iloc[idx], len(val), combined_cost_center.invoice.iloc[idx]/len(val), i])


df = pd.DataFrame(ls, columns=['idx', 'date', 'total_cost', 'number_of_codes', 'cost_per_code', 'cost_center_code'])
#df

In [11]:
df[['date', 'total_cost', 'number_of_codes', 'cost_per_code', 'cost_center_code']].head()

,date,total_cost,number_of_codes,cost_per_code,cost_center_code
0,2018-01,21518.27000,1,21518.27000,540100
1,2018-01,2689.82000,1,2689.82000,548050
2,2018-01,8448.62341,1,8448.62341,Calc
3,2018-01,14030.68000,2,7015.34000,520242
4,2018-01,14030.68000,2,7015.34000,581203


In [12]:
df[df.date == '2019-07']

,idx,date,total_cost,number_of_codes,cost_per_code,cost_center_code
962,594,2019-07,38398.880000,1,38398.880000,540100
963,595,2019-07,5204.960000,1,5204.960000,548050
964,596,2019-07,15217.740160,1,15217.740160,Calc
965,597,2019-07,3105.920000,2,1552.960000,520242
966,597,2019-07,3105.920000,2,1552.960000,581203
967,598,2019-07,625.910000,1,625.910000,554173
968,599,2019-07,1215.083848,1,1215.083848,Calc
969,600,2019-07,11692.780000,2,5846.390000,520241
970,600,2019-07,11692.780000,2,5846.390000,548070
971,601,2019-07,1732.930000,1,1732.930000,554172


In [13]:
# Testing

In [14]:
all_invoices = combined_invoice_detail.copy()
all_invoices['general_ledger'] = np.where(all_invoices['dept'] == 'Contracted Charges', all_invoices['invoice'], all_invoices['general_ledger'])
all_invoices['dept'] = pd.Categorical(all_invoices['dept'], dept_category_order)
all_invoices.general_ledger = np.where(all_invoices['dept'] == 'Contracted Charges', all_invoices['invoice'], all_invoices['general_ledger'])
#jan19 = all_invoices[all_invoices.date == pd.Period('2019-01', 'M')]

In [15]:
personnel_expense_markup = (((all_invoices[all_invoices.dept.isin(total_fixed_personnel_depts +
                                      total_variable_personnel_depts)]
 [['date', 'general_ledger']]
 .groupby('date').sum() * 0.08))
 .assign(dept='personnel_expense_markup')
 .set_index('dept', append=True))

In [16]:
operating_expense_markup = (((all_invoices[all_invoices.dept.isin(total_operating_depts +
                                                                 ['Contracted Charges'])]
 [['date', 'general_ledger']]
 .groupby('date').sum() * 0.04))
 .assign(dept='operating_expense_markup')
 .set_index('dept', append=True))

In [17]:
markup_df = (pd.concat([personnel_expense_markup.reset_index(), operating_expense_markup.reset_index()], ignore_index=True)
             .groupby(['date', 'dept']).sum())

In [18]:
grouped_by_dept = all_invoices.groupby(['date', 'dept']).sum().fillna(markup_df)

In [19]:
grouped_by_dept.groupby('date').sum()

,general_ledger,invoice
date,,
2018-01,429496.609989,13125.25
2018-02,347206.259478,13125.25
2018-03,564891.788457,13125.25
2018-04,401541.261119,13125.25
2018-05,416091.125343,13125.25
2018-06,337195.414165,13125.25
2018-07,427194.771961,13125.25
2018-08,462663.878854,13125.25
2018-09,418524.116998,16625.25


In [20]:
#consumer_fy_plan_group_ordering

['Warehouse Labor', 'Indirect Labor', 'Clerical',
 '?Fixed Payroll?',
 'Warehouse O.T.', 'Indirect Labor O.T.', 'Clerical OT',
 '?Wages & Salaries - Local Alloc?',
 '?Incentives-Cash Basis?',
 '?FICA Expense - Co. Portion?',
 '?Federal Unemployment Insurance?',
 '?State Unemployment Insurance?',
 '?Workers Compensation?',
 '?Group Insurance?',
 '?Other Disc Fringe - 401K?',
 '?Comp Absences-Direct Labor Whs?',
 '?Comp Absences-Indirect Labor W?',
 '?Comp Absences-Clerical Staff?',
 '?Purchased Trans-Motor Carr?',
 '?Other Equipment Lease?',
 '?Depr Exp - Material Handling?',
 '?Depr Exp - Computer Equipment?',
 '?Equipment Maintenance-Other?',
 '?Comp Hardware Subcontr Maint?',
 '?IT Equipment Maintenance?',
 '?Travel - Air/Lodging/Other?',
 '?Business Meal & Entertainment?',
 '?Conferences/Training?',
 '?Temp Direct Labor Whse?',
 '?Employee Safety Programs?',
 '?Professional Fees?',
 '?Other Insurance Expense?',
 '?Office Supplies?',
 '?Supplies-Dock?',
 '?Small Tools/Equip/Other?',
 '?Computer Hardware?',
 '?Communications?',
 '?Employee Awards?',
 '?Miscellaneous Operating?']
# Invoice Groupings of Departments

total_fixed_personnel_depts = ['Supervision', 'Supervision Incentive',
       'Supervision Fringes @ 34.90%',
       'Clerical Fringes @ 32.56%',
       'Indirect Vacation/Holiday ',
       'Indirect Fringes @ 44.61% ST, 18.77% OT']

total_variable_personnel_depts = [
       'Warehouse Vacation/Holiday',
       'Warehouse Fringes @ 47.36% ST, 18.77% OT', 'Temporary Labor']

total_operating_depts = ['Office Supplies ', 'Postage', 'Warehouse Supplies',
       'Maintenance - Warehouse + WH Equipment',
       'Computer Hardware/Software Maintenance', 'Misc Expense ',
       'Communications', 'Employment Related Professional Fees',
       'Insurance', 'Travel Expense', 'Amortization Expense',
       'Equipment Rental', 'Janitorial Expense']

markup_dictionary = {**{f:0.08 for f in total_fixed_personnel_depts},
 **{f:0.08 for f in total_variable_personnel_depts},
 **{f:0.04 for f in total_operating_depts}}

dept_category_order = (total_fixed_personnel_depts + total_variable_personnel_depts +
                       total_operating_depts +
              ['Contracted Charges', 'personnel_expense_markup', 'operating_expense_markup'])

In [21]:
grouped_by_dept['general_ledger'].sum()

8839424.254863001

In [22]:
round(grouped_by_dept['general_ledger'].unstack('date'), 2).to_csv(file_path + 'FY18-19.csv')#.groupby('dept')['general_ledger'].mean()

In [23]:
round(grouped_by_dept.groupby(['date', 'dept'])['general_ledger'].sum(), 2).to_csv(r'C:\Users\Christopher.Robertso\Desktop\raw_reports\invoice_trend_190913.csv')

C:\Users\Christopher.Robertso\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: The signature of `Series.to_csv` was aligned to that of `DataFrame.to_csv`, and argument 'header' will change its default value from False to True: please pass an explicit value to suppress this warning.
  """Entry point for launching an IPython kernel.


In [24]:
round(grouped_by_dept.groupby('date').sum()['general_ledger'], 2)

date
2018-01    429496.61
2018-02    347206.26
2018-03    564891.79
2018-04    401541.26
2018-05    416091.13
2018-06    337195.41
2018-07    427194.77
2018-08    462663.88
2018-09    418524.12
2018-10    571931.44
2018-11    511399.74
2018-12    487362.74
2019-01    389160.24
2019-02    464463.09
2019-03    457866.78
2019-04    435158.64
2019-05    397921.70
2019-06    436531.11
2019-07    435081.02
2019-08    447742.54
Freq: M, Name: general_ledger, dtype: float64

In [25]:
grouped_by_dept

general_ledger   invoice
date    dept                                                              
2018-01 Supervision                                 2.151827e+04      0.00
        Supervision Incentive                       2.689820e+03      0.00
        Supervision Fringes @ 34.90%                8.448623e+03      0.00
        Clerical                                    1.403068e+04      0.00
        Clerical OT                                 1.272430e+03      0.00
        Clerical Fringes @ 32.56%                   4.982693e+03      0.00
        Indirect Labor                              3.133287e+04      0.00
        Indirect Labor O.T.                        -1.827980e+03      0.00
        Indirect Vacation/Holiday                   3.898980e+03      0.00
        Indirect Fringes @ 44.61% ST, 18.77% OT     1.537382e+04      0.00
        Warehouse Labor                             7.097441e+04      0.00
        Warehouse O.T.                             -1.908610e+03      0.00
        Warehouse Vacation/Holiday                  1.591545e+04      0.00
        Warehouse Fringes @ 47.36% ST, 18.77% OT    4.079279e+04      0.00
        Temporary Labor                             1.273258e+05      0.00
        Office Supplies                             7.882000e+01      0.00
        Postage                                     6.769600e+03      0.00
        Warehouse Supplies                          6.252776e-13      0.00
        Maintenance - Warehouse + WH Equipment      7.000000e+03      0.00
        Computer Hardware/Software Maintenance      1.513810e+03      0.00
        Misc Expense                               -2.218250e+03      0.00
        Communications                              4.358790e+03      0.00
        Employment Related Professional Fees        2.643230e+03      0.00
        Insurance                                   1.598000e+03      0.00
        Travel Expense                              2.163270e+03      0.00
        Amortization Expense                        7.478000e+03      0.00
        Equipment Rental                                     NaN       NaN
        Janitorial Expense                                   NaN       NaN
        Contracted Charges                          1.312525e+04  13125.25
        personnel_expense_markup                    2.838561e+04       NaN
...                                                          ...       ...
2019-08 Supervision Incentive                      -1.827873e+04      0.00
        Supervision Fringes @ 34.90%                6.227518e+03      0.00
        Clerical                                    3.028450e+03      0.00
        Clerical OT                                 8.708000e+01      0.00
        Clerical Fringes @ 32.56%                   1.014417e+03      0.00
        Indirect Labor                              1.135326e+04      0.00
        Indirect Labor O.T.                         2.681800e+03      0.00
        Indirect Vacation/Holiday                   1.541310e+03      0.00
        Indirect Fringes @ 44.61% ST, 18.77% OT     6.255642e+03      0.00
        Warehouse Labor                             6.996686e+04      0.00
        Warehouse O.T.                              3.508911e+04      0.00
        Warehouse Vacation/Holiday                  1.398579e+04      0.00
        Warehouse Fringes @ 47.36% ST, 18.77% OT    4.634620e+04      0.00
        Temporary Labor                             1.410042e+05      0.00
        Office Supplies                             7.304800e+02      0.00
        Postage                                     6.088090e+03      0.00
        Warehouse Supplies                          4.645570e+03      0.00
        Maintenance - Warehouse + WH Equipment      1.152410e+03      0.00
        Computer Hardware/Software Maintenance      3.571300e+03      0.00
        Misc Expense                                2.966000e+02      0.00
        Communications                              3.291130e+03      0.00


In [ ]:
## Mapping Customer Invoice to Consumer Report
## 2019-10-02 08:50

In [1]:
import pandas as pd

In [24]:
path = r'my_user_name_path\invoices\consumer_report\Consumer FY20 August Actuals.xlsx'
path2 = r'my_user_name_path\invoices\customer_invoice\fender_invoice_201908.xlsm'

In [7]:
consumer_report_aug19 = pd.read_excel(path, sheet_name='Fender', skiprows=5)

In [30]:
customer_invoice_aug19 = pd.read_excel(path2, sheet_name='Invoice', usecols=[1, 5]).dropna()

In [15]:
list(consumer_report_aug19[['Unnamed: 0', 'Actual']].dropna()['Unnamed: 0'])

['419204 - Distribution Handling Revenue',
 '419216 - Fixed Fees - Management Fees',
 'Net Revenue',
 '520240 - Variable Pay-Direct Whse',
 '520241 - Variable Pay-Indirect Labor Wh',
 '520242 - Variable Pay-Clerical Staff',
 '540100 - Fixed Payroll',
 '554171 - Overtime - Direct Whse',
 '554172 - Overtime - Indirect Labor Whse',
 '554173 - Overtime - Clerical Staff',
 '548050 - Incentives-Cash Basis',
 '583100 - FICA Expense - Co. Portion',
 '583200 - Federal Unemployment Insurance',
 '583300 - State Unemployment Insurance',
 '583400 - Workers Compensation',
 '583700 - Workers Comp Other',
 '582100 - Group Insurance',
 '582700 - Other Disc Fringe - 401K',
 '581201 - Comp Absences-Direct Labor Whs',
 '581202 - Comp Absences-Indirect Labor W',
 '581203 - Comp Absences-Clerical Staff',
 'Salaries and Employee Benefits',
 'Total Salaries and Benefits as a % of Revenue',
 '731630 - Purchased Trans-Motor Carr',
 'Purchased Transportation',
 'Total Purchased Transportation as a % of Revenue',

In [34]:
list(customer_invoice_aug19['Unnamed: 1'])

['Supervision',
 'Supervision Incentive',
 'Supervision Fringes @ 34.90%',
 'Clerical',
 'Clerical OT',
 'Clerical Fringes @ 32.56%',
 'Indirect Labor',
 'Indirect Labor O.T.',
 'Indirect Vacation/Holiday ',
 'Indirect Fringes @ 44.61% ST, 18.77% OT',
 'Warehouse Labor',
 'Warehouse O.T.',
 'Warehouse Vacation/Holiday',
 'Warehouse Fringes @ 47.36% ST, 18.77% OT',
 'Temporary Labor',
 'Office Supplies ',
 'Postage',
 'Warehouse Supplies',
 'Shipping Supplies',
 'Janitorial Expense',
 'Sanitation Expense',
 'Maintenance - Warehouse + WH Equipment',
 'Computer Hardware/Software Maintenance ',
 'Misc Expense ',
 'Equipment Rental',
 'Communications',
 'Employment Related Professional Fees',
 'Out of Scope',
 'Rubbish Removal',
 'Time Keeping System',
 'Insurance',
 'Travel Expense',
 'Amortization Expense',
 'Site Specific Overhead',
 'Infrastructure as a Service (IaaS) monthly charge per agreement',
 'Monthly WMS Support per agreement',
 'Personnel Expense - 8%',
 'Operating Expense - 4%